In [1]:
using Pkg
Pkg.DEFAULT_IO[] = stdout
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [ff71e718] MixedModels v4.4.0 `~/.julia/dev/MixedModels`
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10


In [2]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels
using NLopt
using HighDimMixedModels

In [3]:
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

,period,herd,incid,hsz
,Float64,String,Int8,Int8
1,1.0,H01,2,14
2,2.0,H01,3,12
3,3.0,H01,4,9
4,4.0,H01,0,5
5,1.0,H02,3,22


#### parse random effect in formula

In [4]:
f = @formula(period ~ 0 + incid + hsz + (1|herd))
HMM = highDimMixedModel(f, df, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ HighDimMixedModels /Users/zyxu/.julia/dev/HighDimMixedModels/src/bricks.jl:12


In [5]:
HMM.Z.Z

56×15 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮

In [6]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

sigma = [1.0, 1.0]
negLog = -57.1099018030012
The initial object value is -57.1099018030012
sigma = [1.0, 1.0]
negLog = -57.1099018030012
OPTBL: starting point [1.0, 1.0]
sigma = [1.0, 1.0]
negLog = -57.1099018030012
sigma = [1.75, 1.0]
negLog = -55.97912076973617
sigma = [1.0, 1.75]
negLog = -52.37720093978967
sigma = [0.25, 1.0]
negLog = -64.50473446075652
sigma = [1.0, 0.25]
negLog = -128.22481961167495
sigma = [0.29289321881345254, 0.0]
negLog = Inf
sigma = [0.9469669914110089, 0.19696699141100893]
negLog = -156.76581599673838
sigma = [0.995676667220638, 0.13993745974082217]
negLog = -207.31013755591323
sigma = [0.9952996645647639, 0.06493840728682465]
negLog = -405.77399467791014
sigma = [0.9878315127277438, 0.06424796674686986]
negLog = -410.13655848976134
sigma = [0.987341243774038, 0.05676400814509819]
negLog = -458.32682425399554
sigma = [0.9948025184365511, 0.05600283772361585]
negLog = -463.4181543729141
sigma = [0.9947292842170775, 0.05525642178789495]
negLog = -469.0572593

## debug opt

In [7]:
n = size(HMM.M, 1)
A = hcat(HMM.M.M, HMM.X.X)
P = I - A*inv(transpose(A)*A)*transpose(A)
u,s,v = svd(P)
r = size(HMM.M,2) + size(HMM.X,2)  # simplify: assume fixed effect full rank
#C = randn((n-r),n)
C = transpose(u[:,1:(n-r)]) 
K = C*P
Z = HMM.Z.Z
y = HMM.y
# C can be any full rank matrix with size n,r, e.g. randn(n,r)
print()

In [8]:
A

56×2 Matrix{Float64}:
  2.0  14.0
  3.0  12.0
  4.0   9.0
  0.0   5.0
  3.0  22.0
  1.0  18.0
  1.0  21.0
  8.0  22.0
  2.0  16.0
  0.0  16.0
  2.0  20.0
  2.0  10.0
  0.0  10.0
  ⋮    
  1.0  21.0
  2.0  24.0
  0.0  19.0
  0.0  23.0
 11.0  19.0
  0.0   2.0
  0.0   3.0
  0.0   2.0
  1.0  19.0
  1.0  15.0
  1.0  15.0
  0.0  15.0

#### (a) construct Opt by optsum object

In [9]:
alg = :LN_BOBYQA
verbose = true
## add optsum
# init para
sigma = [1.0,1.0]
lbd = [0.0; 0.0]
optsum = OptSummary(Float64.(sigma), lbd, alg; ftol_rel=(1.0e-12), ftol_abs=(1.0e-8), xtol_rel = 1e-5)
optsum.REML = true

## init opt based on optsum
opt = Opt(optsum)

function negLogLik(sigma::Vector{Float64}, g::Vector{Float64})
    n = length(y)
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    #negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    negLog = 1/2*log(det(K*Sigma*transpose(K))) + 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    #println("OPT: parameter $(sigma) || objective eval $(negLog)")
    @show sigma
    @show negLog

    return negLog
end

println("The initial object value is $(negLogLik(Float64.(sigma), [1.0,1.0]))")

opt.min_objective = negLogLik
optsum.finitial = negLogLik(optsum.initial, [1.0,1.0])  # the second field not useful right now

sigma = [1.0, 1.0]
negLog = 57.1099018030012
The initial object value is 57.1099018030012
sigma = [1.0, 1.0]
negLog = 57.1099018030012


57.1099018030012

In [10]:
if verbose println("OPTBL: starting point $(sigma)") ; end    # to stdout
(optf,optx,ret) = optimize(opt, sigma)

OPTBL: starting point [1.0, 1.0]
sigma = [1.0, 1.0]
negLog = 57.1099018030012
sigma = [1.75, 1.0]
negLog = 55.97912076973617
sigma = [1.0, 1.75]
negLog = 52.37720093978967
sigma = [0.25, 1.0]
negLog = 64.50473446075652
sigma = [1.0, 0.25]
negLog = 128.22481961167495
sigma = [1.510389333579731, 1.4284710320133605]
negLog = 52.78019245866251
sigma = [0.8451353314889427, 1.437431307110896]
negLog = 53.411902302579605
sigma = [1.0, 1.9]
negLog = 52.31808747690741
sigma = [0.822599313455345, 1.9607062304338283]
negLog = 52.490735469672636
sigma = [1.048013829974836, 1.957616595969803]
negLog = 52.30520784011165
sigma = [1.0036098308862278, 2.0180590054596546]
negLog = 52.357982155798666
sigma = [1.0739875442712838, 1.9305681918459645]
negLog = 52.28417164563711
sigma = [1.1350679280066387, 1.887045936358205]
negLog = 52.251667221635145
sigma = [1.2289088557480747, 1.849055393660007]
negLog = 52.22563694469997
sigma = [1.257874889898738, 1.8483895067953622]
negLog = 52.221764542962624
sigma 

(52.21654798673772, [1.3383190003454069, 1.8213128791002569], :FTOL_REACHED)

In [11]:
Sigma = optx[1]*Z*transpose(Z) + optx[2]*diagm(ones(n))
beta = inv(transpose(A)*inv(Sigma)*A)*transpose(A)*inv(Sigma)*y
betaM = beta[1:size(HMM.M,2)]
betaX = beta[(size(HMM.M,2) + 1): size(beta)[1]]

optsum.feval = opt.numevals
optsum.final = optx
optsum.fmin = optf
optsum.returnvalue = ret

if verbose println("got $(round(optf, digits=5)) at $(round.(optx, digits=5)) after $(opt.numevals) iterations (returned $(ret))") ; end

HMM.optsum = optsum

got 52.21655 at [1.33832, 1.82131] after 32 iterations (returned FTOL_REACHED)


|                          |                    |
|:------------------------ |:------------------ |
| **Initialization**       |                    |
| Initial parameter vector | [1.0, 1.0]         |
| Initial objective value  | 57.1099018030012   |
| **Optimizer settings**   |                    |
| Optimizer (from NLopt)   | `LN_BOBYQA`        |
| Lower bounds             | [0.0, 0.0]         |
| `ftol_rel`               | 1.0e-12            |
| `ftol_abs`               | 1.0e-8             |
| `xtol_rel`               | 1.0e-5             |
| `xtol_abs`               | [1.0e-10, 1.0e-10] |
| `initial_step`           | [0.75, 0.75]       |
| `maxfeval`               | -1                 |
| `maxtime`                | -1.0               |
| **Result**               |                    |
| Function evaluations     | 32                 |
| Final parameter vector   | [1.3383, 1.8213]   |
| Final objective value    | 52.2165            |
| Return code              | `FTOL_REACHED`     |


In [13]:
beta

2-element Vector{Float64}:
 -0.2938738291611581
  0.1317420744001128

In [14]:
optsum.feval

32

In [15]:
f = optsum.feval
f = 3
f

3

In [16]:
optsum.feval

32

In [17]:
betaM

1-element Vector{Float64}:
 -0.2938738291611581

In [18]:
betaX

1-element Vector{Float64}:
 0.1317420744001128

#### (b) construct opt by Opt(:LN_BOBYQA, 2)

In [19]:
using NLopt
alg = :LN_BOBYQA # :LN_COBYLA
verbose = true
## add optsum
# init para
sigma = [1.0,1.0]
lbd = [0.0; 0.0]
optsum = OptSummary(Float64.(sigma), lbd, alg; ftol_rel=(1.0e-12), ftol_abs=(1.0e-8), xtol_rel = 1e-5)
optsum.REML = true

## init opt based on optsum
#opt = Opt(optsum)

## init opt based on Opt function
opt = Opt(:LN_BOBYQA, 2)

NLopt.ftol_rel!(opt, optsum.ftol_rel) # relative criterion on objective
NLopt.ftol_abs!(opt, optsum.ftol_abs) # absolute criterion on objective
NLopt.xtol_rel!(opt, optsum.xtol_rel) # relative criterion on parameter values
NLopt.xtol_abs!(opt, optsum.xtol_abs) # absolute criterion on parameter values
NLopt.lower_bounds!(opt, lbd)
NLopt.maxeval!(opt, optsum.maxfeval)
NLopt.maxtime!(opt, optsum.maxtime)
optsum.initial_step = NLopt.initial_step(opt, optsum.initial, similar(lbd))

2-element Vector{Float64}:
 0.75
 0.75

In [20]:
opt

Opt(LN_BOBYQA, 2)

In [21]:
function negLogLik(sigma::Vector{Float64}, g::Vector{Float64})
    n = length(y)
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = 1/2*log(det(K*Sigma*transpose(K))) + 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    #println("OPT: parameter $(sigma) || objective eval $(negLog)")
    @show sigma
    @show negLog

    return negLog
end

println("The initial object value is $(negLogLik(Float64.(sigma), [1.0,1.0]))")

opt.min_objective = negLogLik
optsum.finitial = negLogLik(optsum.initial, [1.0,1.0])  # the second field not useful right now

if verbose println("OPTBL: starting point $(sigma)") ; end    # to stdout
(optf,optx,ret) = optimize(opt, sigma)

sigma = [1.0, 1.0]
negLog = 57.1099018030012
The initial object value is 57.1099018030012
sigma = [1.0, 1.0]
negLog = 57.1099018030012
OPTBL: starting point [1.0, 1.0]
sigma = [1.0, 1.0]
negLog = 57.1099018030012
sigma = [1.75, 1.0]
negLog = 55.97912076973617
sigma = [1.0, 1.75]
negLog = 52.37720093978967
sigma = [0.25, 1.0]
negLog = 64.50473446075652
sigma = [1.0, 0.25]
negLog = 128.22481961167495
sigma = [1.510389333579731, 1.4284710320133605]
negLog = 52.78019245866251
sigma = [0.8451353314889427, 1.437431307110896]
negLog = 53.411902302579605
sigma = [1.0, 1.9]
negLog = 52.31808747690741
sigma = [0.822599313455345, 1.9607062304338283]
negLog = 52.490735469672636
sigma = [1.048013829974836, 1.957616595969803]
negLog = 52.30520784011165
sigma = [1.0036098308862278, 2.0180590054596546]
negLog = 52.357982155798666
sigma = [1.0739875442712838, 1.9305681918459645]
negLog = 52.28417164563711
sigma = [1.1350679280066387, 1.887045936358205]
negLog = 52.251667221635145
sigma = [1.22890885574

(52.21654798673772, [1.3383190003454069, 1.8213128791002569], :FTOL_REACHED)

In [22]:
if verbose println("got $(round(optf, digits=5)) at $(round.(optx, digits=5)) after $(opt.numevals) iterations (returned $(ret))") ; end

got 52.21655 at [1.33832, 1.82131] after 32 iterations (returned FTOL_REACHED)


#### (c) No REML

In [23]:
n = size(HMM.M, 1)
A = hcat(HMM.M.M, HMM.X.X)
#P = I - A*inv(transpose(A)*A)*transpose(A)
#u,s,v = svd(P)
r = size(HMM.M,2) + size(HMM.X,2)  # simplify: assume fixed effect full rank
#C = randn((n-r),n)
#C = transpose(u[:,1:(n-r)]) 
#K = C*P
Z = HMM.Z.Z
y = HMM.y
# C can be any full rank matrix with size n,r, e.g. randn(n,r)
print()

$$
\mathbf{y} \text { is } N_{n}(\mathbf{X} \boldsymbol{\beta}, \mathbf{\Sigma}), \quad \text { where } \quad \mathbf{\Sigma}= \sigma_{z}^{2}
\mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} 
$$

The objective function is:
$$
\frac{n-r}{2} \ln (2 \pi)-\frac{1}{2} \ln \left| \Sigma \right|-\frac{1}{2} (\mathbf{y}-A \beta)^{\prime}\left(\mathbf{K} \Sigma \mathbf{K}^{\prime}\right)^{-1}(\mathbf{y}-A \beta)
$$

__In this part I set parameter in optimization to be sigma(2 element vector) and beta(2 element vector, this specific to the data)__

In [24]:
alg = :LN_BOBYQA
verbose = true
## add optsum
# init para
para = [1.0, 1.0, 3.0, 4.0] # sigma and beta
lbd = [0.0, 0.0, 0.0, 0.0]
optsum = OptSummary(Float64.(para), lbd, alg; ftol_rel=(1.0e-12), ftol_abs=(1.0e-8), xtol_rel = 1e-5)
optsum.REML = true

## init opt based on optsum
opt = Opt(optsum)

function negLogLik(para::Vector{Float64}, g::Vector{Float64})
    n = length(y)
    Sigma = para[1]*Z*transpose(Z) + para[2]*diagm(ones(n))
    negLog = 1/2*log(det(K*Sigma*transpose(K))) + 1/2*transpose(y-A*para[3:4])*transpose(K)*inv(K*Sigma*transpose(K))*K*(y-A*para[3:4])
    #println("OPT: parameter $(para) || objective eval $(negLog)")
    @show para
    @show negLog

    return negLog
end

println("The initial object value is $(negLogLik(Float64.(para), [1.0,1.0]))")

opt.min_objective = negLogLik
optsum.finitial = negLogLik(optsum.initial, [1.0,1.0, 3.0, 4.0])  # the second field not useful right now

para = [1.0, 1.0, 3.0, 4.0]
negLog = 57.109901803001115
The initial object value is 57.109901803001115
para = [1.0, 1.0, 3.0, 4.0]
negLog = 57.109901803001115


57.109901803001115

In [25]:
if verbose println("OPTBL: starting point $(para)") ; end    # to stdout
(optf,optx,ret) = optimize(opt, para)

OPTBL: starting point [1.0, 1.0, 3.0, 4.0]
para = [1.0, 1.0, 3.0, 4.0]
negLog = 57.109901803001115
para = [1.75, 1.0, 3.0, 4.0]
negLog = 55.979120769736056
para = [1.0, 1.75, 3.0, 4.0]
negLog = 52.37720093978956
para = [1.0, 1.0, 5.25, 4.0]
negLog = 57.10990180300113
para = [1.0, 1.0, 3.0, 7.0]
negLog = 57.10990180300106
para = [0.25, 1.0, 3.0, 4.0]
negLog = 64.50473446075647
para = [1.0, 0.25, 3.0, 4.0]
negLog = 128.2248196116742
para = [1.0, 1.0, 0.75, 4.0]
negLog = 57.109901803001044
para = [1.0, 1.0, 3.0, 1.0]
negLog = 57.10990180300117
para = [1.5103893335797347, 1.4284710320133616, 2.999999999999983, 4.00000000000003]
negLog = 52.78019245866247
para = [0.8451353314889464, 1.4374313071108966, 2.999999999999997, 4.000000000000005]
negLog = 53.411902302579506
para = [1.0, 1.9, 3.0, 4.0]
negLog = 52.31808747690738
para = [0.8170269316937663, 1.9409524880147997, 2.9999999999999725, 4.00000000000005]
negLog = 52.49733594190326
para = [1.0, 1.957616595969803, 2.855958510075492, 4.0]
neg

(52.2165480346999, [1.3380757261571867, 1.8213661917781347, 3.8985497031710326, 5.522474166381298], :FTOL_REACHED)

In [26]:
Sigma = optx[1]*Z*transpose(Z) + optx[2]*diagm(ones(n))
beta = inv(transpose(A)*inv(Sigma)*A)*transpose(A)*inv(Sigma)*y
betaM = beta[1:size(HMM.M,2)]
betaX = beta[(size(HMM.M,2) + 1): size(beta)[1]]

optsum.feval = opt.numevals
optsum.final = optx
optsum.fmin = optf
optsum.returnvalue = ret

if verbose println("got $(round(optf, digits=5)) at $(round.(optx, digits=5)) after $(opt.numevals) iterations (returned $(ret))") ; end

HMM.optsum = optsum

got 52.21655 at [1.33808, 1.82137, 3.89855, 5.52247] after 56 iterations (returned FTOL_REACHED)


|                          |                                      |
|:------------------------ |:------------------------------------ |
| **Initialization**       |                                      |
| Initial parameter vector | [1.0, 1.0, 3.0, 4.0]                 |
| Initial objective value  | 57.109901803001115                   |
| **Optimizer settings**   |                                      |
| Optimizer (from NLopt)   | `LN_BOBYQA`                          |
| Lower bounds             | [0.0, 0.0, 0.0, 0.0]                 |
| `ftol_rel`               | 1.0e-12                              |
| `ftol_abs`               | 1.0e-8                               |
| `xtol_rel`               | 1.0e-5                               |
| `xtol_abs`               | [1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10] |
| `initial_step`           | [0.75, 0.75, 2.25, 3.0]              |
| `maxfeval`               | -1                                   |
| `maxtime`                | -1.0                                 |
| **Result**               |                                      |
| Function evaluations     | 56                                   |
| Final parameter vector   | [1.3381, 1.8214, 3.8985, 5.5225]     |
| Final objective value    | 52.2165                              |
| Return code              | `FTOL_REACHED`                       |
